Main Break Sensitivity Analysis
Once the water main asset prediction/assessment function has been performed this script can be applied to review input sensitivity 
It is basically repeating a method to print results in the notebook using pandas data frame table summaries
ie query and display results using this technology

Each field of interest can be reviewed with a total count of breaks, total feet of pipe for the field value, then another table showing the value and the number of breaks per 1000ft of pipe having that value in order to gain understanding of the sensitivity of that value group to likelhood of break.

in the results we are looking at relative high values for breaks per 1000 feet with shape length minimums having good statistically represetative groups

In [2]:
watermains = r"Water_Main_202110_breakpoints"

#make a selection for potable 

arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)




In [ ]:
#what is the sensitivity of soil corrosive property to likelihood of main break?
#use dissolve + summary fields

arcpy.management.Dissolve(watermains, "WaterMainD_Corrosive", "SoilCorrosionValue", "Shape_Length SUM;BreakCount SUM", "MULTI_PART", "DISSOLVE_LINES")
arcpy.management.CalculateField("WaterMainD_Corrosive", "BreaksPerThousandFeet", "!SUM_BreakCount!/!SUM_Shape_Length!*1000", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

#look at results - this is one way

In [ ]:
#what is the sensitivity of material to likelihood of main break?

watermains = r"Water_Main_202110_breakpoints"

arcpy.management.Dissolve(watermains, "WaterMainD_Corrosive", "SoilCorrosionValue", "Shape_Length SUM;BreakCount SUM", "MULTI_PART", "DISSOLVE_LINES")
arcpy.management.CalculateField("WaterMainD_Corrosive", "BreaksPerThousandFeet", "!SUM_BreakCount!/!SUM_Shape_Length!*1000", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

#it would be better to have the results display in this jupyter result

In [35]:
#what is the sensitivity of soil corrosive property to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['SoilCorrosionValue', 'BreakCount', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['SoilCorrosionValue']).sum()
print(table)
table = df.groupby(['SoilCorrosionValue']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#table = df.groupby(['SoilCorrosionValue']).sum().apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)

#if medium corrosivity is more likely to break than high corrisivity should we be scoring this smiliar weight to age or material?

#display(HTML(table.to_html()))

                    BreakCount   Shape_Length
SoilCorrosionValue                           
0                           90   85346.872321
3                          107   77682.954590
5                          263  228931.449734
SoilCorrosionValue
0    1.054520
3    1.377394
5    1.148816
dtype: float64


In [13]:
#what is the sensitivity of pipe material to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['Material', 'BreakCount', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Material']).sum()
print(table)
table = df.groupby(['Material']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)



#display(HTML(table.to_html()))

          BreakCount   Shape_Length
Material                           
CAS              244  116529.710294
COP               11    5422.997240
DIP              153  245247.910027
GP                 3    1148.247339
NIL                0      20.995332
PE                 3     157.210684
PVC               24    3511.871730
RCP                3    2118.909497
SP                 7    1454.823820
TTE               12   16348.600683
Material
CAS     2.093887
COP     2.028399
DIP     0.623859
GP      2.612678
NIL     0.000000
PE     19.082673
PVC     6.833963
RCP     1.415823
SP      4.811579
TTE     0.734008
dtype: float64


In [16]:
#what is the sensitivity of pipe diameter to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['Diameter', 'BreakCount', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Diameter']).sum()
print(table)
table = df.groupby(['Diameter']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)



#display(HTML(table.to_html()))

          BreakCount   Shape_Length
Diameter                           
0.75               0     237.547121
1.00               1    1319.738293
1.50               1     281.399305
2.00              17    5992.734224
3.00               0     386.546257
4.00              18    9237.827205
6.00             166   74390.946845
8.00             165  120377.749501
10.00              2    2950.385394
12.00             67  119322.054231
14.00              0    4012.187715
16.00              7   28293.819610
20.00              7    2429.418638
24.00              9   22439.434414
30.00              0     289.487891
Diameter
0.75     0.000000
1.00     0.757726
1.50     3.553669
2.00     2.836769
3.00     0.000000
4.00     1.948510
6.00     2.231454
8.00     1.370685
10.00    0.677878
12.00    0.561506
14.00    0.000000
16.00    0.247404
20.00    2.881348
24.00    0.401080
30.00    0.000000
dtype: float64


In [14]:
#what is the sensitivity of age to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['Install_Era', 'BreakCount', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Install_Era']).sum()
print(table)
table = df.groupby(['Install_Era']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)



#display(HTML(table.to_html()))

             BreakCount   Shape_Length
Install_Era                           
1850-1899             2    1988.778455
1900-1909             1     367.871921
1910-1919             1    4818.134838
1920-1929            14   12750.983405
1930-1939            11    5105.312850
1940-1949             9    7818.775325
1950-1959            86   60460.122046
1960-1969            71   32983.518407
1970-1979           127   65532.607314
1980-1989            65  121314.141580
1990-1999            37   51573.208831
2000-2009            30   27032.938777
2010-2019             6     214.882898
Install_Era
1850-1899     1.005642
1900-1909     2.718337
1910-1919     0.207549
1920-1929     1.097955
1930-1939     2.154618
1940-1949     1.151075
1950-1959     1.422425
1960-1969     2.152590
1970-1979     1.937967
1980-1989     0.535799
1990-1999     0.717427
2000-2009     1.109757
2010-2019    27.922185
dtype: float64


In [15]:
#what is the sensitivity of age and material to likelihood of main break?

import pandas as pd
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['Install_Era', 'Material', 'BreakCount', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Install_Era', 'Material']).sum()
print(table)
table = df.groupby(['Install_Era', 'Material']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)
from IPython.display import display, HTML
print(table)
#display(table)



#display(HTML(table.to_html()))

                      BreakCount   Shape_Length
Install_Era Material                           
1850-1899   CAS                2    1988.778455
1900-1909   CAS                1     343.918963
            PVC                0      23.952958
1910-1919   CAS                1    4813.302354
            GP                 0       0.871175
            SP                 0       3.961308
1920-1929   CAS               14   12066.750748
            GP                 0      11.981977
            PVC                0     124.352581
            TTE                0     547.898100
1930-1939   CAS               11    4839.229536
            TTE                0     266.083315
1940-1949   CAS                4     603.516226
            GP                 0       2.083055
            PVC                0       5.931706
            TTE                5    7207.244338
1950-1959   CAS               73   47031.966117
            DIP                0    1704.852083
            GP                 2     707

In [27]:
import pandas as pd
from IPython.display import display, HTML
arcpy.management.SelectLayerByAttribute(watermains, "NEW_SELECTION", "WATERTYPE = 'Potable' And MAINTBY NOT IN (-1)", None)

fc = watermains

fields = ['Install_Era', 'Material', 'Diameter', 'BreakCount', 'Shape_Length'] #Change to match your input

df = pd.DataFrame.from_records(data=arcpy.da.SearchCursor(fc,fields),columns=fields)
table = df.groupby(['Install_Era', 'Material', 'Diameter']).sum()
display(HTML(table.to_html()))
#table2 = df.groupby(['Install_Era', 'Material', 'Diameter']).apply(lambda x: x['BreakCount'].sum()/x['Shape_Length'].sum()*1000)

#print(table2)
#display(table2)

#display(HTML(table2.to_html()))  #this expression needs a field name for the lambda field I think